# Simple chat-bot example

In [4]:
!pip install pymorphy2
!pip install stop_words
!pip install annoy
!pip install gensim
!pip install tqdm

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=6b9397990b8ea20f8517762a7479ee4b6f733cd24d8cc10bb5b7575499fb121b
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words
     |████████████████████████████████| 655kB 2.8MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp36-cp36m-linux_x86_64.whl size=390359 sha256=05d8b587bc83e5dccdfb73e2af93ee9bafdaab1040b4bfcc6a7adc769d5a20c1
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


In [26]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import sys

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
#assert False

#Small preprocess of the answers

question = None
written = False

with open("/content/drive/My Drive/prepared_answers.txt", "w") as fout:
    with open("/content/drive/My Drive/Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [8]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [37]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("/content/drive/My Drive/Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

In [38]:
sentences = [i for i in sentences if len(i) > 2]

In [39]:
sentences[0]

['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']

In [40]:
modelW2V = Word2Vec(sentences=sentences, size=50, window=5, min_count=1)

In [41]:
modelFT = FastText(sentences=sentences, size=50, min_count=1, window=5)

In [45]:
w2v_index = annoy.AnnoyIndex(50 ,'angular')
ft_index = annoy.AnnoyIndex(50 ,'angular')

index_map = {}
counter = 0

with open("/content/drive/My Drive/prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(50)
        vector_ft = np.zeros(50)
        for word in question:
            if word in modelW2V:
                vector_w2v += modelW2V[word]
                n_w2v += 1
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

True

In [46]:
w2v_index.get_nns_by_vector(np.zeros(50), 3)

[261, 2235, 2285]

In [47]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(50)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [48]:
TEXT = "главный вопрос жизни, вселенной и всего такого"

In [49]:
import warnings
warnings.filterwarnings("ignore")

In [50]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Вы уверены, что Вам правду ответят?. \n',
 '<p> Они сражались за свою жизнь и родину и за все </p>. \n',
 'На то она и классическая литература, которая не утратит своего значения никогда. Пусть даже пройдет 20 или 30 лет, но книги будут актуальны. классическая литература - это как правило очень большие произведения и в них рассматриваются очень много вопросов, которые модные и сейчас и на которых мы до сих пор ищем ответы. "Что делать"" Чернышевского, или "отцы и дети", "А судьи кто? " Пушкина, "Мастер и Маргарита (Булгакова) - проблема взаимоотношений, религий и любви. "Кому на Руси жить хорошо? " Некрасова. "Война и мир" Толстого - это вообще энциклопедия актуальности.. \n']

In [51]:
get_response(TEXT, ft_index, modelFT, index_map)

['В раю канечно климат полудше но зато в аду более приятное общество (с) непомню кто...<br>;-). \n',
 'за. ОТМОРОЗКИ ОЧЕНЬ БОЯТСЯ СМЕРТИ.. \n',
 'Помощник президента по техническим вопросам. <br>А если серьезно, то скорее всего хороший автослесарь будет на первых местах.. \n']

In [52]:
from gensim.test.utils import get_tmpfile

file_name = get_tmpfile("/content/drive/My Drive/chat_bot/FastText.model")
modelFT.save(file_name)

In [53]:
file_name = get_tmpfile("/content/drive/My Drive/chat_bot/modelW2V.model")
modelW2V.save(file_name)

In [56]:
import json
with open('/content/drive/My Drive/chat_bot/index_map.txt', 'w') as file:
    json.dump(index_map, file)